In [1]:
import pandas as pd
import seaborn as sns
from pycorenlp import StanfordCoreNLP
import numpy as np
import ast
from sklearn import preprocessing


# Doesn't include 
## Ticker	Date	Open	High	Low	Close	Volume	Dividends	Stock Splits

In [2]:
df=pd.read_csv('./data/fdb.csv')
df['news_sentiment_score']= df['news_sentiment_score'].fillna(0)
df['amount_of_articles']= df['amount_of_articles'].fillna(0)
df['headline']=df['headline'].fillna('[]')
df['headline']=df['headline'].apply(lambda x: ast.literal_eval(x))
df

,Ticker,Date,Open,Close,Volume,headline,news_sentiment_score,amount_of_articles,Same_day_move,from_previous_day
0,TSLA,2021-01-04,719.460022,729.770020,48638200,"[2 Tesla Analysts Break Down Fundamentals, Val...",1.71,55.0,1,0
1,TSLA,2021-01-05,723.659973,735.109985,32245200,[Tesla Gets Regulatory Nod To Begin Sales In I...,1.76,41.0,1,0
2,TSLA,2021-01-06,758.489990,755.979980,44700000,[Tesla Option Traders Are Dumping Massive Amou...,1.83,48.0,0,1
3,TSLA,2021-01-07,777.630005,816.039978,51498900,"[Elon Musk, Jeff Bezos Battling It Out For The...",2.04,45.0,1,1
4,TSLA,2021-01-08,856.000000,880.020020,75055500,"[Tesla To Launch $25,000 EV Sedan In China By ...",1.71,41.0,1,1
...,...,...,...,...,...,...,...,...,...,...
319,AMD,2021-04-23,80.209999,82.760002,49194000,"[What to Expect From AMD This Earnings Season,...",1.87,23.0,1,1
320,AMD,2021-04-26,83.349998,85.410004,57594500,[Chipmaker stocks are going to be ‘the key’ to...,1.95,22.0,1,1
321,AMD,2021-04-27,85.669998,85.209999,61909900,"[Dow Jones Futures: Microsoft, Google, AMD Lea...",1.93,59.0,0,1
322,AMD,2021-04-28,88.849998,84.019997,108920300,"[U.S. stocks end lower, despite Powell vow to ...",2.00,53.0,0,1


In [3]:
df.columns

Index(['Ticker', 'Date', 'Open', 'Close', 'Volume', 'headline',
       'news_sentiment_score', 'amount_of_articles', 'Same_day_move',
       'from_previous_day'],
      dtype='object')

In [4]:
df1=df[['Ticker']]
le = preprocessing.LabelEncoder()

df1 = df1.apply(le.fit_transform)

df1 = pd.concat([df1,df[['Open', 'Close', 'Volume','news_sentiment_score', 
                         'amount_of_articles', 
#                          'Same_day_move',
                         'from_previous_day']]], 
                axis=1)
df1.head()

# For 'Same_day_move','from_previous_day'
# down is 0
# up and equal is 1


,Ticker,Open,Close,Volume,news_sentiment_score,amount_of_articles,from_previous_day
0,3,719.460022,729.770020,48638200,1.71,55.0,0
1,3,723.659973,735.109985,32245200,1.76,41.0,0
2,3,758.489990,755.979980,44700000,1.83,48.0,1
3,3,777.630005,816.039978,51498900,2.04,45.0,1
4,3,856.000000,880.020020,75055500,1.71,41.0,1


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.preprocessing import FunctionTransformer


numeric_features = ['Open', 'Close','Volume','news_sentiment_score', 
                     'amount_of_articles', 'Same_day_move']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Ticker']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', LogisticRegression())])

In [39]:
X = df.drop(['Date','headline','from_previous_day'],1)
y = df.from_previous_day

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,stratify=df['Ticker'])

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



def models(X_train,y_train):
    
    
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', LogisticRegression()),
                     ])
    clf.fit(X_train, y_train)
    log=clf.score(X_test, y_test)
    
    clf1 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('svc', SVC(kernel='linear',random_state=0)),
                     ])
    clf1.fit(X_train, y_train)
    svc_line = clf1.score(X_test, y_test)
    
    clf2 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('svc', SVC(kernel='rbf',random_state=0)),
                     ])
    clf2.fit(X_train, y_train)
    svc_rbf = clf2.score(X_test, y_test)
    
    
    clf3 = Pipeline(steps=[('preprocessor', preprocessor),
                      ('KNN', KNeighborsClassifier()),
                     ])
    clf3.fit(X_train, y_train)
    knn=clf3.score(X_test, y_test)
    
    
    clf4 = Pipeline(steps=[('preprocessor', preprocessor),
                          ('GNB', GaussianNB()),
                         ])
    clf4.fit(X_train, y_train)
    gauss=clf4.score(X_test, y_test)    
    
    
    
    clf5 = Pipeline(steps=[('preprocessor', preprocessor),
                          ('tree', DecisionTreeClassifier(criterion = 'entropy',random_state=0)),
                         ])
    clf5.fit(X_train, y_train)
    tree=clf5.score(X_test, y_test) 
    
    
    clf6 = Pipeline(steps=[('preprocessor', preprocessor),
                          ('tree', RandomForestClassifier(n_estimators=10, criterion = 'entropy',random_state=0)),
                         ])
    clf6.fit(X_train, y_train)
    forest=clf6.score(X_test, y_test)
    

    print('[0]Logistic Regression Accuracy: ' ,clf.score(X_train,y_train)*100)
    print(f'{log}')
    print()
    print('[1]SVC_Line Regression Accuracy: ' ,clf1.score(X_train,y_train)*100)
    print(f'{svc_line}')
    print()
    print('[2]SVC_RBf Regression Accuracy: ' ,clf2.score(X_train,y_train)*100)
    print(f'{svc_rbf}')
    print()
    print('[3]KNN Regression Accuracy: ' ,clf3.score(X_train,y_train)*100)
    print(f'{knn}')
    print()
    print('[4]Gauss Regression Accuracy: ' ,clf4.score(X_train,y_train)*100)
    print(f'{gauss}')
    print()
    print('[5]Tree Regression Accuracy: ' ,clf5.score(X_train,y_train)*100)
    print(f'{tree}')
    print()
    print('[6]Forest Regression Accuracy: ' ,clf6.score(X_train,y_train)*100)
    print(f'{forest}')
    print()
    
    return log, svc_line, svc_rbf, knn, gauss, tree, forest

In [18]:
models(X_train,y_train)

[0]Logistic Regression Accuracy:  60.08230452674898
0.5432098765432098

[1]SVC_Line Regression Accuracy:  58.8477366255144
0.5679012345679012

[2]SVC_RBf Regression Accuracy:  60.49382716049383
0.6049382716049383

[3]KNN Regression Accuracy:  69.1358024691358
0.6049382716049383

[4]Gauss Regression Accuracy:  54.73251028806584
0.5061728395061729

[5]Tree Regression Accuracy:  100.0
0.48148148148148145

[6]Forest Regression Accuracy:  97.94238683127571
0.5185185185185185



(0.5432098765432098,
 0.5679012345679012,
 0.6049382716049383,
 0.6049382716049383,
 0.5061728395061729,
 0.48148148148148145,
 0.5185185185185185)

In [38]:
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.model_selection import GridSearchCV

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier' , LogisticRegression())])

param_grid = [
    {
         'classifier' : [LogisticRegression()],
         'classifier__penalty' : ['none','l1', 'l2','elasticnet'],
         'classifier__C' : [100, 10, 1.0,0.1,0.01],
         'classifier__solver' : ['newton-cg','liblinear']
    },
    {
        'classifier':[RidgeClassifier()],
        'classifier__alpha':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    },
    {
         'classifier' : [KNeighborsClassifier()],
         'classifier__n_neighbors' : [1,3,5,7,9,11,13,15,17,19,21],
         'classifier__metric' : ['euclidean','manhattan','minkowski']
    },
    {
        'classifier' : [SVC()],
        'classifier__kernel' : ['linear'],
        'classifier__C' : [1]
    },
    {
        'classifier' : [BaggingClassifier()],
        'classifier__n_estimators':[10, 100, 1000]

    },
    {
        'classifier' : [RandomForestClassifier()],
        'classifier__n_estimators':[10, 100, 1000],
        'classifier__max_features' : ['sqrt', 'log2']
    },
    {
        'classifier' : [GradientBoostingClassifier()],
        'classifier__n_estimators':[10, 100, 1000],
        'classifier__learning_rate':[0.001, 0.01, 0.1],
        'classifier__subsample' : [0.5, 0.7, 1.0],
       'classifier__max_depth' : [3, 7, 9]
        
    },
    {
        'classifier' : [GaussianNB()],
        'classifier__var_smoothing': np.logspace(0,-9, num=100)
    }
]

# Create grid search object

clf = GridSearchCV(estimator=pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

In [52]:
print("Best: %f using %s" % (best_clf.best_score_, best_clf.best_params_))

TypeError: must be real number, not Pipeline

<hr>

# Includes 
## Ticker	Date	Open	High	Low	Close	Volume	Dividends	Stock Splits


In [77]:
fdb1=pd.DataFrame()
for i in df['Ticker'].unique():
    dff=pd.read_csv(f'./data/{i}.csv')
    dff.insert(0, "Ticker", i)
    fdb1=pd.concat([fdb1,dff])
fdb1.reset_index(drop=True,inplace=True)

In [78]:
fdb1

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,TSLA,2021-01-04,719.460022,744.489990,717.190002,729.770020,48638200,0.0,0
1,TSLA,2021-01-05,723.659973,740.840027,719.200012,735.109985,32245200,0.0,0
2,TSLA,2021-01-06,758.489990,774.000000,749.099976,755.979980,44700000,0.0,0
3,TSLA,2021-01-07,777.630005,816.989990,775.200012,816.039978,51498900,0.0,0
4,TSLA,2021-01-08,856.000000,884.489990,838.390015,880.020020,75055500,0.0,0
...,...,...,...,...,...,...,...,...,...
319,AMD,2021-04-23,80.209999,83.300003,79.959999,82.760002,49194000,0.0,0
320,AMD,2021-04-26,83.349998,85.900002,82.699997,85.410004,57594500,0.0,0
321,AMD,2021-04-27,85.669998,87.150002,85.129997,85.209999,61909900,0.0,0
322,AMD,2021-04-28,88.849998,89.199997,83.919998,84.019997,108920300,0.0,0


In [80]:
df_new=df[['news_sentiment_score','amount_of_articles','Same_day_move','from_previous_day']]

In [81]:
df_new

,news_sentiment_score,amount_of_articles,Same_day_move,from_previous_day
0,1.71,55.0,1,0
1,1.76,41.0,1,0
2,1.83,48.0,0,1
3,2.04,45.0,1,1
4,1.71,41.0,1,1
...,...,...,...,...
319,1.87,23.0,1,1
320,1.95,22.0,1,1
321,1.93,59.0,0,1
322,2.00,53.0,0,1


In [84]:
new_df=pd.concat([fdb1, df_new], axis=1)
new_df

,Ticker,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,news_sentiment_score,amount_of_articles,Same_day_move,from_previous_day
0,TSLA,2021-01-04,719.460022,744.489990,717.190002,729.770020,48638200,0.0,0,1.71,55.0,1,0
1,TSLA,2021-01-05,723.659973,740.840027,719.200012,735.109985,32245200,0.0,0,1.76,41.0,1,0
2,TSLA,2021-01-06,758.489990,774.000000,749.099976,755.979980,44700000,0.0,0,1.83,48.0,0,1
3,TSLA,2021-01-07,777.630005,816.989990,775.200012,816.039978,51498900,0.0,0,2.04,45.0,1,1
4,TSLA,2021-01-08,856.000000,884.489990,838.390015,880.020020,75055500,0.0,0,1.71,41.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,AMD,2021-04-23,80.209999,83.300003,79.959999,82.760002,49194000,0.0,0,1.87,23.0,1,1
320,AMD,2021-04-26,83.349998,85.900002,82.699997,85.410004,57594500,0.0,0,1.95,22.0,1,1
321,AMD,2021-04-27,85.669998,87.150002,85.129997,85.209999,61909900,0.0,0,1.93,59.0,0,1
322,AMD,2021-04-28,88.849998,89.199997,83.919998,84.019997,108920300,0.0,0,2.00,53.0,0,1


In [88]:
X = new_df.drop(['Date','from_previous_day'],1)
y = new_df.from_previous_day

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, stratify=new_df['Ticker'])

In [91]:
numeric_features = ['Open', 'Close','High','Low','Volume','Dividends','Stock Splits',
                    'news_sentiment_score', 'amount_of_articles', 'Same_day_move']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Ticker']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regression', LogisticRegression())])

In [94]:
from sklearn.model_selection import GridSearchCV

pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier' , LogisticRegression())])

param_grid = [
    {
         'classifier' : [LogisticRegression()],
         'classifier__penalty' : ['none','l1', 'l2','elasticnet'],
         'classifier__C' : [100, 10, 1.0,0.1,0.01],
         'classifier__solver' : ['newton-cg','liblinear']
    },
    {
        'classifier':[RidgeClassifier()],
        'classifier__alpha':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    },
    {
         'classifier' : [KNeighborsClassifier()],
         'classifier__n_neighbors' : [1,3,5,7,9,11,13,15,17,19,21],
         'classifier__metric' : ['euclidean','manhattan','minkowski']
    },
    {
        'classifier' : [SVC()],
        'classifier__kernel' : ['linear'],
        'classifier__C' : [1]
    },
    {
        'classifier' : [BaggingClassifier()],
        'classifier__n_estimators':[10, 100, 1000]

    },
    {
        'classifier' : [RandomForestClassifier()],
        'classifier__n_estimators':[10, 100, 1000,3000],
        'classifier__max_features' : ['sqrt', 'log2']
    },
    {
        'classifier' : [GradientBoostingClassifier()],
        'classifier__n_estimators':[10, 100, 1000],
        'classifier__learning_rate':[0.001, 0.01, 0.1],
        'classifier__subsample' : [0.5, 0.7, 1.0],
       'classifier__max_depth' : [3, 7, 9]
        
    },
    {
        'classifier' : [GaussianNB()],
        'classifier__var_smoothing': np.logspace(0,-9, num=100)
    }
]

# Create grid search object

clf = GridSearchCV(estimator=pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 276 candidates, totalling 1380 fits


/Users/home/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.55595238        nan        nan 0.55578231 0.55170068 0.55170068
        nan        nan 0.55595238        nan        nan 0.56828231
 0.55994898 0.55994898        nan        nan 0.55595238        nan
        nan 0.57670068 0.57653061 0.57653061        nan        nan
 0.55595238        nan        nan 0.55170068 0.55586735 0.57236395
        nan        nan 0.55595238        nan        nan 0.42380952
 0.55578231 0.57210884        nan        nan 0.54753401 0.54345238
 0.54761905 0.55170068 0.55578231 0.55986395 0.55986395 0.56394558
 0.55986395 0.56819728 0.50255102 0.53911565 0.55578231 0.52661565
 0.55552721 0.55561224 0.53920068 0.54761905 0.55578231 0.56003401
 0.55153061 0.44489796 0.54744898 0.53911565 0.57210884 0.56811224
 0.52270408 0.55170068 0.56411565 0.55986395 0.56794218 0.55144558
 0.50255102 0.53911565 0.55578231 0.526615

In [95]:
print("Best: %f using %s" % (best_clf.best_score_, best_clf.best_params_))

Best: 0.617432 using {'classifier': GradientBoostingClassifier(n_estimators=10, subsample=0.7), 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 10, 'classifier__subsample': 0.7}


In [93]:
print("Best: %f using %s" % (best_clf.best_score_, best_clf.best_params_))

Best: 0.605017 using {'classifier': RandomForestClassifier(max_features='log2', n_estimators=1000), 'classifier__max_features': 'log2', 'classifier__n_estimators': 1000}
